## 01. Imports and Inputs

In [50]:

import reading_adjusting as ra

import pandas as pd
import numpy as np
import os

import sklearn
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import shap
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode,  plot

In [2]:
root= 'C:/Users/PauloAlves/Documents/Pessoais/MercadoLibre/'

## 02. Folder Structure

In [3]:
save_names = [
    root_folder,
    root_folder + "01. Data/",
    root_folder + "02. Results/",
]

for save in save_names:
    if not os.path.isdir(save):
        try:
            os.mkdir(save)
        except OSError:
            print("\nCreation of the %s failed" % save)
            raise OSError
        else:
            print("\nSuccessfully created %s " % save)


## 03. Webscrapping and DataStrucuture

In [4]:
################# 01. Tv4k
### 01.A - Reading tv4k results and creating a structured pandas dataframe
final_tv, tv_error = ra.create_product_table('tv%204k')
final_tv = final_tv.loc[final_tv['domain_id'] ==  'MLA-TELEVISIONS' ,:]

### 01.B - capturing id tables
final_tv_id, final_tv_error = ra.create_id_table(df = final_tv, id_name = 'id', filter_prod= 'MLA-TELEVISIONS' )

### 01.C - writing Tv4k data
final_tv.to_csv(root_folder + "01. Data/dados_tv.csv")
final_tv_id.to_csv(root_folder + "01. Data/dados_tv_id.csv")

Product Extraction: [##########] 100% Done...      
Id Extraction: [##########] 100% Done...      


In [9]:
print("final_tv has shape",final_tv.shape," with ",len(tv_error),"errors")
print("final_tv_id has shape",final_tv_id.shape," with ",len(final_tv_error),"errors")

final_tv has shape (693, 197)  with  0 errors
final_tv_id has shape (688, 191)  with  5 errors


In [6]:
################# 02. Cellphone
### 02.A - Reading cellphone results and creating a structured pandas dataframe
final_cp, cp_error = ra.create_product_table('celular')
final_cp = final_cp.loc[final_cp['domain_id'] ==  'MLA-CELLPHONES' ,:]

### 02.B - capturing id tables
final_cp_id, final_cp_error = ra.create_id_table(df = final_cp, id_name = 'id', filter_prod= 'MLA-CELLPHONES' )

### 02.C - writing cellphone data
final_cp.to_csv(root_folder + "01. Data/dados_celular.csv")
final_cp_id.to_csv(root_folder + "01. Data/dados_celular_id.csv")


Product Extraction: [##########] 100% Done...      
Id Extraction: [##########] 100% Done...      


In [10]:
print("final_cp has shape",final_cp.shape," with ",len(cp_error),"errors")
print("final_cp_id has shape",final_cp_id.shape," with ",len(final_cp_error),"errors")

final_cp has shape (960, 194)  with  0 errors
final_cp_id has shape (956, 326)  with  2 errors


In [8]:
################# 03. Notebook
### 03.A - Reading notebook results and creating a structured pandas dataframe
final_nb, np_error= ra.create_product_table('notebook')
final_nb = final_nb.loc[final_nb['domain_id'] ==  'MLA-NOTEBOOKS' ,:]

### 03.B - capturing id tables
final_nb_id, final_np_error = ra.create_id_table(df = final_nb, id_name = 'id', filter_prod= 'MLA-NOTEBOOKS' )

### 03.C - writing notebook data
final_nb.to_csv(root_folder + "01. Data/dados_notebook.csv")
final_nb_id.to_csv(root_folder + "01. Data/dados_notebook_id.csv")


Product Extraction: [##########] 100% Done...      
Id Extraction: [##########] 100% Done...      


In [11]:
print("final_nb has shape",final_nb.shape," with ",len(np_error),"errors")
print("final_nb_id has shape",final_nb_id.shape," with ",len(final_np_error),"errors")

final_nb has shape (1000, 203)  with  0 errors
final_nb_id has shape (993, 225)  with  7 errors


In [14]:
################# 04. Smartwatch
### 04.A - Reading smartwatch results and creating a structured pandas dataframe
final_sw, sw_error = ra.create_product_table('smartwatch')
final_sw = final_sw.loc[final_sw['domain_id'] ==  'MLA-SMARTWATCHES' ,:]

### 04.B - capturing id tables
final_sw_id, final_sw_error = ra.create_id_table(df = final_sw, id_name = 'id', filter_prod= 'MLA-SMARTWATCHES' )

### 04.C - writing smartwatch data
final_sw.to_csv(root_folder + "01. Data/dados_sw.csv")
final_sw_id.to_csv(root_folder + "01. Data/dados_sw_id.csv")


Product Extraction: [##########] 100% Done...      
Id Extraction: [##########] 100% Done...      


In [15]:
print("final_sw has shape",final_sw.shape," with ",len(sw_error),"errors")
print("final_sw_id has shape",final_sw_id.shape," with ",len(final_sw_error),"errors")

final_sw has shape (998, 170)  with  0 errors
final_sw_id has shape (995, 445)  with  2 errors


# 04. Analises

##### Estos análisis se centran en comprender un poco el comportamiento de los datos y también en tratar de tener algunos conocimientos comerciales

### 04.A - Seller Badges 

In [124]:

u,c = np.unique([j for i in final_tv['seller__tags'] for j in i], return_counts = True)
tags_df = pd.DataFrame({"Tags":u,"Counts":c,"Percentage":c/final_tv.shape[0]})
tags_df.sort_values('Counts',ascending = False, inplace = True)


fig = go.Figure([go.Bar(x=tags_df['Tags'], y=tags_df['Percentage'])])
fig.update_layout(yaxis=dict(title='Porcentaje'),
                  xaxis=dict(title="Etiqueta"),
                 title = 'Seller Badges - Porcentaje de vendedores por etiqueta ')
fig.show()

### 04.B - Descontos por tipo de produto

In [131]:

fig = go.Figure()

fig2 = go.Figure()
dict_fig2 = {}
nombres = {'final_tv': 'TV4k','final_cp':'teléfono celular', 'final_nb': 'computadora portátil','final_sw':'SmartWatch'}
for table in ['final_tv','final_cp','final_nb','final_sw']:
    temp_data = globals()[table]
    temp_data = temp_data[['price','original_price','attributes_0__value_name','shipping__free_shipping']]
    temp_data['discount_b'] = temp_data['original_price'].isnull()
    temp_data['original_price2'] = temp_data['original_price']
    temp_data.loc[ temp_data['discount_b'] == True,'original_price2'] = temp_data.loc[ temp_data['discount_b'] == True, 'price']
    temp_data['discount_p'] = 1 - temp_data['price'] / temp_data['original_price2'] 
    temp_data['discount_bin'] = pd.cut(temp_data['discount_p'],np.arange(0,105,5)/100, include_lowest=True).astype("object")
    
    aux = temp_data.groupby(['discount_bin']).size()
    aux = aux/aux.sum()
    
    fig.add_trace(go.Bar(x = [str(ind) for ind in aux.index]
                         ,y = aux
                      #  ,histnorm='percent'
                        ,name=nombres[table]
                        ,opacity = 0.7))
    
    fig.update_layout(yaxis=dict(title='Porcentaje'),
                  xaxis=dict(title="Producto & Rango de descuento"),
                 title = 'Porcentaje de descuentos por tipo de producto')
    
    dict_fig2[table] = (temp_data['discount_p'] > 0).sum()/temp_data.shape[0]
fig.show()


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

### 04.C - Desconto médio por tipo de produto

In [132]:
fig2.add_trace(go.Bar(x = [nombres[k] for k in dict_fig2.keys()]
                         ,y = [np.round(dict_fig2[k],2) for k in dict_fig2.keys()]
                      #  ,histnorm='percent'
                        ,name=nombres[table]
                        ,opacity = 0.7))
fig2.update_layout(yaxis=dict(title='Descuento'),
                  xaxis=dict(title="Producto"),
                 title = 'Porcentaje de descuentos por tipo de producto')
fig2.show()

### 04.D - Condições dos produtos por tipo

In [133]:

fig = go.Figure()
for table in ['final_tv','final_cp','final_nb','final_sw']:
    temp_data = globals()[table]
    temp_data = temp_data[['condition']]
    
    aux = temp_data.groupby(['condition']).size()
    aux = aux/aux.sum()
    
    fig.add_trace(go.Bar(x = [str(ind) for ind in aux.index]
                         ,y = aux
                      #  ,histnorm='percent'
                        ,name=nombres[table]
                        ,opacity = 0.7))
fig.update_layout(yaxis=dict(title='Porcentaje'),
                  xaxis=dict(title="Condición"),
                 title = 'Porcentaje de nuevos y usados por tipo del producto')
fig.show()

### 04.E - Preço médio por  para TVs

In [134]:

fig = go.Figure()
for table in ['final_tv']:
    temp_data = globals()[table]
    temp_data = temp_data.loc[temp_data['condition'] == 'new', ['attributes_0__value_name','price']]
    
    aux = temp_data.groupby(['attributes_0__value_name'])['price'].mean()
    aux = aux.sort_values(ascending = False)
    
    fig.add_trace(go.Bar(x = [str(ind) for ind in aux.index]
                         ,y = aux
                      #  ,histnorm='percent'
                        ,name=nombres[table]
                        ,opacity = 0.7))
fig.update_layout(yaxis=dict(title='Precio Medio'),
                  xaxis=dict(title="BRAND"),
                 title = 'Precio medio por marca de televisión')
fig.show()

### 04.F - Oferta gratuita de envio por tipo de produto

In [135]:

fig = go.Figure()
fig2 = go.Figure()
for table in ['final_tv','final_cp','final_nb','final_sw']:
    temp_data = globals()[table]
    temp_data = temp_data[['price','shipping__free_shipping']]

    
    aux = temp_data.groupby(['shipping__free_shipping']).size()
    aux = aux/aux.sum()
    
    aux2 = temp_data.groupby(['shipping__free_shipping'])['price'].mean()
    
    
    fig.add_trace(go.Bar(x = [str(ind) for ind in aux.index]
                         ,y = aux
                      #  ,histnorm='percent'
                        ,name=nombres[table]
                        ,opacity = 0.7))
    
    fig2.add_trace(go.Bar(x = [str(ind) for ind in aux2.index]
                         ,y = aux2
                      #  ,histnorm='percent'
                        ,name=nombres[table]
                        ,opacity = 0.7))

fig.update_layout(yaxis=dict(title='Porcentaje'),
                  xaxis=dict(title="Free-Shipping?"),
                 title = 'Porcentaje de ofertas con envío gratuito por tipo de producto')
fig.show()

### 04.G - Preço médio por tipo de produto com/sem envio grátis 

In [136]:
fig2.update_layout(yaxis=dict(title='Precio'),
                  xaxis=dict(title="Free-Shipping?"),
                 title = 'Precio Medio de vendas con envío gratuito por tipo de producto')

fig2.show()

### 04.H - Tags sobre imagens ruins

In [164]:

temp_data = pd.DataFrame()
for table in ['final_tv','final_cp','final_nb','final_sw']:
    temp_data = temp_data.append(globals()[table])
    
temp_data = temp_data[['seller__id','seller__registration_date','tags']]
temp_data['poor'] = temp_data.apply(lambda x: len([ t for t in x['tags'] if t.startswith("poor")]), axis = 1)

## poor_quality_thumbnail, poor_quality_picture

gb = temp_data.groupby(['seller__id']).size().reset_index()
gb.columns = ['seller__id', 'qtt']
temp_data = temp_data.merge( gb, on = ['seller__id' ])
temp_data = temp_data[['poor','qtt']].drop_duplicates()

temp_data.loc[temp_data['poor'] == 1]

(temp_data['poor'] == 1).sum()

fig = go.Figure()
fig.add_trace(go.Box(y = temp_data['qtt'],
                     x = temp_data['poor']

                            ,name=nombres[table]
                            ,opacity = 0.7)) 

fig.update_layout(yaxis=dict(title='Numero de publicaciones por vendedor'),
                  xaxis=dict(title="Número de etiquetas en nota de mala imagen"),
                 title = 'Relación entre cantidad de publicaciones y etiquetas de mala calidad')
fig.show()


### 04.I - Quantidade x Qualidad x Tiempo desde el registro

In [140]:

temp_data = pd.DataFrame()
for table in ['final_tv','final_cp','final_nb','final_sw']:
    temp_data = temp_data.append(globals()[table])

temp_data = temp_data[['seller__id','seller__registration_date','tags']]
temp_data['poor'] = temp_data.apply(lambda x: len([ t for t in x['tags'] if t.startswith("poor")]), axis = 1)
temp_data['good'] = temp_data.apply(lambda x: len([ t for t in x['tags'] if t.startswith("good")]), axis = 1)
temp_data['register_year'] = pd.to_datetime(temp_data['seller__registration_date']).dt.year
temp_data['register_time'] = 2021 - temp_data['register_year'] 

gb = temp_data.groupby(['seller__id',]).size().reset_index()

gb.columns = ['seller__id', 'qtt']
temp_data = temp_data.merge( gb, on = ['seller__id' ])
temp_data = temp_data[['poor','qtt','register_time']].drop_duplicates()

from sklearn.preprocessing import MinMaxScaler, StandardScaler


mms = StandardScaler()
mms.fit(temp_data[['qtt','register_time']])
temp_data[['qtt_mms','register_time_mms']] = pd.DataFrame(mms.transform(temp_data[['qtt','register_time']]), index = temp_data.index)


mask = temp_data['poor'] > 0
fig = go.Figure(data=go.Scatter(x=temp_data.loc[mask ,'register_time'],
                                y=temp_data.loc[mask , 'qtt'],
                                mode='markers',
                                marker=dict(
                                                size=16,
                                                color=temp_data.loc[mask ,'poor'], #set color equal to a variable
                                                 # one of plotly colorscales
                                                showscale=True
                                            )))

fig.update_layout(yaxis=dict(title='Numero de publicaciones por vendedor'),
                  xaxis=dict(title="Número de años desde el registro"),
                 title = 'Relación entre cantidad de publicaciones, etiquetas de mala calidad y tiempo desde el registro')
                            
fig.show()

In [177]:

temp_data = pd.DataFrame()
for table in ['final_tv','final_cp','final_nb','final_sw']:
    temp_data = temp_data.append(globals()[table])

temp_data = temp_data[['seller__id','seller__registration_date','tags']]
temp_data['poor'] = temp_data.apply(lambda x: len([ t for t in x['tags'] if t.startswith("poor")]), axis = 1)
temp_data['good'] = temp_data.apply(lambda x: len([ t for t in x['tags'] if t.startswith("good")]), axis = 1)
temp_data['register_year'] = pd.to_datetime(temp_data['seller__registration_date']).dt.year
temp_data['register_time'] = 2021 - temp_data['register_year'] 

gb = temp_data.groupby(['seller__id',]).size().reset_index()

gb.columns = ['seller__id', 'qtt']
temp_data = temp_data.merge( gb, on = ['seller__id' ])
temp_data = temp_data[['poor','qtt','register_time']].drop_duplicates()

from sklearn.preprocessing import MinMaxScaler, StandardScaler


mms = StandardScaler()
mms.fit(temp_data[['qtt','register_time']])
temp_data[['qtt_mms','register_time_mms']] = pd.DataFrame(mms.transform(temp_data[['qtt','register_time']]), index = temp_data.index)

fig = go.Figure()
for i in [0,1,2]:
    mask = temp_data['poor'] == i
    gb = temp_data.loc[mask ,:].groupby('register_time')['qtt'].mean()
    
    
    fig.add_trace(go.Bar(x = gb.index,
                                    y= gb.values,
                         name = 'poor: '+str(i)
                                    ))

fig.update_layout(yaxis=dict(title='Numero médio de publicaciones por vendedor'),
                  xaxis=dict(title="Número de años desde el registro"),
                 title = 'Relación entre cantidad de publicaciones, etiquetas de mala calidad y tiempo desde el registro')
                            
fig.show()

### 04.J - Mapeo de vendedores basado en criterios negativos.

In [143]:

temp_data = pd.DataFrame()
for table in ['final_tv','final_cp','final_nb','final_sw']:
    temp_data = temp_data.append(globals()[table][['seller__id','seller__seller_reputation__metrics__cancellations__rate'
                                 ,'seller__seller_reputation__metrics__delayed_handling_time__rate'
                                 ,'seller__seller_reputation__metrics__claims__rate']])

temp_data = temp_data.drop_duplicates()
temp_data.columns = ["id","cancellation","delay","claims"]
temp_data = temp_data.loc[temp_data['cancellation'] < 0.5,:]
    
trace1 = go.Scatter3d(x=temp_data['cancellation'],
                                   y=temp_data['delay'],
                                   z=temp_data['claims'],
                                   mode='markers')


y_max = max(temp_data['delay'])
y_min = min(temp_data['delay'])
x_max = max(temp_data['cancellation'])
x_min = min(temp_data['cancellation'])
z_max = max(temp_data['claims'])
z_min = min(temp_data['claims'])

    
x_lines = list([0.02 ,0.02 ,None, x_max, x_min, None  , 0.02 , 0.02 , None]  )
y_lines = list([y_max,y_min,None, 0.1  ,  0.1 , None  , 0.1  , 0.1  , None])
z_lines = list([0.02 ,0.02 ,None, 0.02 , 0.02 , None  , z_max, z_min, None])
trace2 = go.Scatter3d(
    x=x_lines,
    y=y_lines,
    z=z_lines,
    mode='lines',
    name='lines'
)

fig = go.Figure(data=[trace1, trace2])
fig.update_layout(
                 title = 'Demora x Cancelamientos x Reclamaciones')
fig.update_layout(scene = dict(
                    xaxis_title='Porcentaje de cancelamientos',
                    yaxis_title='Porcentaje de demoras',
                    zaxis_title='Porcentaje de reclamaciones'))

fig.show()